<span style='color:red'> NOTE: You can only pass the lab, when you provide both code and markdown </span>

Use Code for your analysis
Use Markdown to document and elaborate on your findings, conclusions, assertions, etc.

# DS_ML_I_P1: Dataset creation from raw data 
Provided is a list of Excel-Files that stem from a radar measurement using an array of 15 Antennas and a frequency sweep. Another Excel sheet provides information on the type of object that should be detected and its orientation.

The overall task is to load the data into **a single dataframe**, add the **proper information on object**, **orientation** and **the name of the image that shows the object** (which is not provided here)


## 1. Load the data and check proper loading
Load all the data into a single dataframe so that
* The name of the file is a separat column
* Only the first five columns and all rows per Sheet tab should be integrated (15 Tabs in total, one per Antenna)
* Sheet tab name should be the major index in a multiindex column dataframe
* Tab column names should be the minor index
* After this dataframe has been created the object information, orientation and image name should be added as separate columns by integrating the information from the specific excel sheet.

In [56]:
import pandas as pd
import glob
import os
import numpy as np


In [57]:
pd.options.display.max_columns=1000
pd.options.display.max_rows=20


The code loads measurement data from multiple Excel files, each containing measurement data from 15 antennas, and organizes it into one structured DataFrame. It fulfills the task requirements by:

* Combining all files into a single DataFrame,
* Adding the filename as extra information,
* Using a MultiIndex for the columns (major: sheet name / minor: five measurement values from each sheet).

In [58]:
%%time
file_paths = glob.glob("P1b/Measurements_8_April_2023_IMP-SIMO/*.xls")

dfs =[]

for file in file_paths:
    file_data = []
    sheets = pd.read_excel(file, sheet_name=None, usecols=[0,1,2,3,4])

    for sheet_name, df in sheets.items():
        multi_columns = [
            np.repeat(sheet_name,len(df.columns)),
            df.columns.to_list()
        ]
        df.columns = pd.MultiIndex.from_arrays(multi_columns, names=['major','minor'])
        file_data.append(df)

    df["filename"] = os.path.splitext(os.path.basename(file))[0]
    dfs.append(pd.concat(file_data, axis=1))

df_measurement = pd.concat(dfs, ignore_index=True)

CPU times: total: 9.67 s
Wall time: 9.98 s


The object information was read into a DataFrame from the measurement protocol file. Each column name was then translated into English.

In [59]:
oi = pd.read_excel("P1b/Messprotokoll_18_04_2023.xlsx", skiprows=6, usecols="C:H").rename(columns={
    "Messung": "measurement", 
    "Gegenstand": "object", 
    "Postion": "position", 
    "Dateienname ": "filename", 
    "Bild ": "picture",
    "Anmerkungen": "comments"
})

The column filename in object information dataframe (`oi`) is set as key. Based on that key, each row's filename in the measurement dataframe (`df_measurement`) is then mapped to the additional information in the object information DataFrame (`oi`). This process enriches the measurement dataframe with the corresponding object, orientation, and image name.

In [60]:
oi['filename'] = oi['filename'].astype(str).str.strip()
oi = oi.set_index('filename')

In [61]:
df_measurement['filename'] = df_measurement['filename'].astype(str).str.strip()

df_measurement['object'] = df_measurement['filename'].map(oi['object'])
df_measurement['orientation'] = df_measurement['filename'].map(oi['position'])
df_measurement['image_name'] = df_measurement['filename'].map(oi['picture'])

## 2. Print some statistics and analyze

### Statistic Analysis Result
* Ingesamt wurden 3000 Zeilen geteilt in 79 Spalten eingelesen. Es gibt 46 Spalten mit DataType float64, 30 Spalten mit DataType int64, und 3 Spalten mit DataType als object
* Die Spalte image_name wurde mit datatype als float64 gelesen, anstatt als object. Transformation ist vermütlich nötig für die später richtige Darstellung der Daten
* 120 Zeilen enthalten missing values. Die meisten missing values sind in den Spalten `orientation` und `image_name`. In der Information sieht man dass die beiden Zeilen nur 2880 entries haben. 3000 Zeilen - 2880 entries = 120 Zeilen mit fehlenden Entries.

In [62]:
df_measurement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 79 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   (ANT 1, DAC Value)       3000 non-null   float64
 1   (ANT 1, DAC Value RAW)   3000 non-null   int64  
 2   (ANT 1, Magnitute)       3000 non-null   float64
 3   (ANT 1, Phase)           3000 non-null   float64
 4   (ANT 1, Frequency)       3000 non-null   int64  
 5   (ANT 2, DAC Value)       3000 non-null   float64
 6   (ANT 2, DAC Value RAW)   3000 non-null   int64  
 7   (ANT 2, Magnitute)       3000 non-null   float64
 8   (ANT 2, Phase)           3000 non-null   float64
 9   (ANT 2, Frequency)       3000 non-null   int64  
 10  (ANT 3, DAC Value)       3000 non-null   float64
 11  (ANT 3, DAC Value RAW)   3000 non-null   int64  
 12  (ANT 3, Magnitute)       3000 non-null   float64
 13  (ANT 3, Phase)           3000 non-null   float64
 14  (ANT 3, Frequency)      

In [63]:
df_measurement.describe()

major        ANT 1                                                        \
minor    DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -29.440857   -11.037441  2.450000e+09   
std       0.727482    346.419787    21.308478   100.516179  0.000000e+00   
min      -1.576100    800.000000   -76.132400  -179.746000  2.450000e+09   
25%      -0.956600   1095.000000   -49.023000   -98.424450  2.450000e+09   
50%      -0.337100   1390.000000   -34.079850   -24.726700  2.450000e+09   
75%       0.282400   1685.000000    -5.197550    71.956325  2.450000e+09   
max       0.901900   1980.000000     2.967040   179.945000  2.450000e+09   

major        ANT 2                                                        \
minor    DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.584241    -8.454822  2.450000e+09   
std       0.727482    346.419787    20.660701   101.662437  0.000000e+00   
min      -1.576100    800.000000   -78.798000  -179.855000  2.450000e+09   
25%      -0.956600   1095.000000   -47.094225  -104.927500  2.450000e+09   
50%      -0.337100   1390.000000   -33.717600    -0.022706  2.450000e+09   
75%       0.282400   1685.000000    -5.459223    80.736000  2.450000e+09   
max       0.901900   1980.000000     2.600070   179.882000  2.450000e+09   

major        ANT 3                                                        \
minor    DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.496339     6.162484  2.450000e+09   
std       0.727482    346.419787    19.896935   105.454240  0.000000e+00   
min      -1.576100    800.000000   -65.850600  -179.841000  2.450000e+09   
25%      -0.956600   1095.000000   -46.240275   -93.691275  2.450000e+09   
50%      -0.337100   1390.000000   -33.768350    20.151350  2.450000e+09   
75%       0.282400   1685.000000    -6.185160    95.470150  2.450000e+09   
max       0.901900   1980.000000     1.951980   179.968000  2.450000e+09   

major        ANT 4                                                        \
minor    DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -27.625137    -5.295042  2.450000e+09   
std       0.727482    346.419787    20.533735   102.950631  0.000000e+00   
min      -1.576100    800.000000   -66.213900  -179.830000  2.450000e+09   
25%      -0.956600   1095.000000   -47.068525  -108.048500  2.450000e+09   
50%      -0.337100   1390.000000   -32.086750     0.879465  2.450000e+09   
75%       0.282400   1685.000000    -4.311635    87.005050  2.450000e+09   
max       0.901900   1980.000000     3.010070   179.962000  2.450000e+09   

major        ANT 5                                                        \
minor    DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.000000  3000.000000  3.000000e+03   
mean     -0.337100   1390.000000   -28.421452   -14.991958  2.450000e+09   
std       0.727482    346.419787    20.865768   101.811997  0.000000e+00   
min      -1.576100    800.000000   -72.225700  -179.865000  2.450000e+09   
25%      -0.956600   1095.000000   -48.134000  -103.428500  2.450000e+09   
50%      -0.337100   1390.000000   -31.816800   -35.516350  2.450000e+09   
75%       0.282400   1685.000000    -4.999288    70.339400  2.450000e+09   
max       0.901900   1980.000000     2.312730   179.475000  2.450000e+09   

major        ANT 6                                                        \
minor    DAC Value DAC Value RAW    Magnitute        Phase     Frequency   
count  3000.000000   3000.000000  3000.0000

In [64]:
print(df_measurement['image_name'].dtype)
print(df_measurement['image_name'].unique())
print(df_measurement['image_name'].min())
print(df_measurement['image_name'].max())

float64
[nan  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48.]
1.0
48.0


In [66]:
df_missing = df_measurement[df_measurement.isna().any(axis=1)]
df_missing

major     ANT 1                                                   ANT 2  \
minor DAC Value DAC Value RAW  Magnitute    Phase   Frequency DAC Value   
0       -1.5761           800 -51.029200 -32.2202  2450000000   -1.5761   
1       -1.5341           820 -51.478400 -30.6490  2450000000   -1.5341   
2       -1.4921           840 -51.622100 -33.3354  2450000000   -1.4921   
3       -1.4501           860 -51.706600 -33.2258  2450000000   -1.4501   
4       -1.4081           880 -52.007200 -27.1376  2450000000   -1.4081   
..          ...           ...        ...      ...         ...       ...   
115      0.7339          1900  -0.520329 -12.7426  2450000000    0.7339   
116      0.7759          1920  -0.495056 -11.3609  2450000000    0.7759   
117      0.8179          1940  -0.489968 -13.0398  2450000000    0.8179   
118      0.8599          1960  -0.512845 -13.5566  2450000000    0.8599   
119      0.9019          1980  -0.555737 -12.3917  2450000000    0.9019   

major                                                   ANT 3                \
minor DAC Value RAW  Magnitute    Phase   Frequency DAC Value DAC Value RAW   
0               800 -49.357300 -36.4674  2450000000   -1.5761           800   
1               820 -49.250800 -40.9706  2450000000   -1.5341           820   
2               840 -49.206500 -40.7474  2450000000   -1.4921           840   
3               860 -49.481300 -37.5041  2450000000   -1.4501           860   
4               880 -49.256800 -39.9416  2450000000   -1.4081           880   
..              ...        ...      ...         ...       ...           ...   
115            1900   0.519030  40.8771  2450000000    0.7339          1900   
116            1920   0.528720  40.7776  2450000000    0.7759          1920   
117            1940   0.515985  41.3485  2450000000    0.8179          1940   
118            1960   0.485382  40.3722  2450000000    0.8599          1960   
119            1980   0.436453  40.2864  2450000000    0.9019          1980   

major                                     ANT 4                          \
minor  Magnitute    Phase   Frequency DAC Value DAC Value RAW Magnitute   
0     -48.255700 -29.1957  2450000000   -1.5761           800 -49.06800   
1     -48.196800 -30.4962  2450000000   -1.5341           820 -49.31200   
2     -48.300400 -34.2268  2450000000   -1.4921           840 -49.18750   
3     -48.205800 -33.0781  2450000000   -1.4501           860 -49.15730   
4     -48.414000 -34.8080  2450000000   -1.4081           880 -49.52880   
..           ...      ...         ...       ...           ...       ...   
115     0.280231  75.5205  2450000000    0.7339          1900   1.92190   
116     0.260036  76.0463  2450000000    0.7759          1920   1.93043   
117     0.287292  75.3413  2450000000    0.8179          1940   1.96533   
118     0.326058  74.3957  2450000000    0.8599          1960   1.89895   
119     0.275221  75.6491  2450000000    0.9019          1980   1.92688   

major                          ANT 5                                    \
minor    Phase   Frequency DAC Value DAC Value RAW  Magnitute    Phase   
0     -34.9213  2450000000   -1.5761           800 -49.537400 -32.5722   
1     -33.9601  2450000000   -1.5341           820 -50.832700 -27.6044   
2     -37.3867  2450000000   -1.4921           840 -50.897200 -28.8071   
3     -35.4936  2450000000   -1.4501           860 -51.943800 -27.6148   
4     -39.2850  2450000000   -1.4081           880 -52.452800 -29.7870   
..         ...         ...       ...           ...        ...      ...   
115    37.2372  2450000000    0.7339          1900   0.140287 -59.0291   
116    36.8665  2450000000    0.7759          1920   0.166123 -60.4332   
117    37.5122  2450000000    0.8179          1940   0.167973 -60.2283   
118    36.5344  2450000000    0.8599          1960   0.150695 -60.6437   
119    37.5294  2450000000    0.9019          1980   0.112695 -58.4123   

major                 ANT 6                              

### Questions for the Prof:

* soll man das DataFrame einfach so lassen oder so man Transformation von DataType/Formate durchführen?
* es gibt mehrere Excel-Datei zu den Messprotokolle. Welche ist relevant?
* was bedeutet die Objekte "Ohne/Mit Kamera"? Müssen sie entfernt oder irgendwo anders gelagert werden?

### Transformationen

## 3. Visualize the data
* Scatter Plot
* Box Plot
* Histogram

## 4. Conclusion
